In [2]:
import re
import random
import torch.nn as nn
import torch
import numpy as np

In [3]:
def read_file(file_name):
    with open(file_name, 'r') as f:
        data = f.read()
    return data

In [4]:
raw_data = read_file('shakespeare.txt')

In [5]:
def remove_non_alpha_characters(data):
    data = data.lower()
    # use regex to remove all non-alphanumeric characters
    data = re.sub(r'[^a-zA-Z\s]', '', data)
    # use regex to remove all whitespace characters
    data = re.sub(r'\s+', ' ', data)
    return data

def return_unique(data):
    unique = set(data)
    return list(unique)

def remove_stopwords(data):
    stopwords = ['a', 'an', 'the', 'and', 'or', 'but', 'if', 'then', 'else', 'when', 'at', 'from', 'by', 'on', 'off', 'for', 'in', 'out', 'over', 'to', 'into', 'with', ""]
    data = [word for word in data if word not in stopwords]
    return data

In [6]:
data = remove_non_alpha_characters(raw_data)
data = data.split(" ")
data = remove_stopwords(data)
unique_words = return_unique(data)

In [7]:
unique_dict = {word: i for i, word in enumerate(unique_words)}

# def one_hot_encode(words):
# words = ["boy", "car", "man"]

def one_hot_encode(words):
    length = len(words.keys())
    encoded_words = {}
    for key, value in words.items():
        one_hot = np.zeros(length)
        one_hot[value] = 1
        tensor = torch.from_numpy(one_hot).to(torch.int64)
        encoded_words[key] = tensor

    return encoded_words

encoded_data = one_hot_encode(unique_dict)

In [8]:
def return_list_without_a_value(data, value):
    return [x for x in data if x != value]



window_size = 5
dataset = []
sample_data = data

for i, val in enumerate(sample_data):
    if i > len(sample_data) - window_size:
        break
    sub = sample_data[i:i+window_size]
    included = return_list_without_a_value(sub, val)
    for target in included:
        dataset.append((unique_dict[val],unique_dict[target]))
    


    

In [49]:
batch_size = 10
n_iters = 3000
num_epochs = 100
num_epochs = int(num_epochs)
# create a train_loader that will randomly generate examples forever

train_loader = torch.utils.data.DataLoader(dataset=dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

In [ ]:
class SkipGramModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGramModel, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.u_embeddings = nn.Embedding(vocab_size, embedding_dim, sparse=True)
        self.v_embeddings = nn.Embedding(vocab_size, embedding_dim, sparse=True)
        self.init_emb()

    def init_emb(self):
        initrange = 0.1
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.uniform_(-initrange, initrange)

    def forward(self, pos_u, pos_v, neg_v):
        # reshape the input tensors to have the right dimensions
        emb_u = self.u_embeddings(pos_u).view(-1, 1, self.embedding_dim).squeeze()
        emb_v = self.v_embeddings(pos_v).view(-1, self.embedding_dim).squeeze()
        # get the dot product of the two embeddings using the bmm function
        score = torch.bmm(emb_u.unsqueeze(1), emb_v.unsqueeze(2)).squeeze()
        score = torch.sigmoid(score)
        neg_emb_v = self.v_embeddings(neg_v).view(-1, self.embedding_dim, 5)
        neg_score = torch.bmm(emb_u.unsqueeze(1), neg_emb_v).squeeze()
        neg_score = torch.sigmoid(neg_score)
        return score, neg_score

    def get_dict_embeddings(self):
        return self.u_embeddings.weight.data.cpu().numpy()
    
    def get_embedding_from_word(self, word):
        index = unique_dict[word]
        return self.u_embeddings.weight.data[index]
    
    def get_embedding_from_index(self, index):
        return self.u_embeddings.weight.data[index]

    def save_embedding(self, id2word, file_name):
        embedding = self.input_embeddings()
        fout = open(file_name, 'w')
        fout.write('{} {}\n'.format(len(id2word), self.embedding_dim))
        for wid, w in id2word.items():
            e = ' '.join(map(lambda x: str(x), embedding[wid]))
            fout.write('{} {}\n'.format(w, e))
        fout.close()
    
    def import_embeddings(self, file_name):
        fin = open(file_name, 'r')
        n, d = map(int, fin.readline().split())
        embedding = np.zeros((n, d))
        word2id = {}
        for line in fin:
            tokens = line.rstrip().split(' ')
            word2id[tokens[0]] = len(word2id)
            embedding[word2id[tokens[0]]] = list(map(float, tokens[1:]))
        return embedding, word2id


embedding_dim = 300
window_size = 5

dictionary_length = len(unique_words)

model = SkipGramModel(dictionary_length, embedding_dim)

In [204]:




criterion = nn.BCELoss()
learning_rate = 0.05
optimizer = torch.optim.SparseAdam(model.parameters(), lr=learning_rate)

loss_sum = 0

negative_sample_length = 5

pos_u_data = torch.ones(batch_size)
neg_v_data = torch.zeros(batch_size*negative_sample_length)
concat_data = torch.cat([pos_u_data, neg_v_data], dim=0)
print(concat_data)

epochs = 15
for epoch in range(epochs):
    for i, (x, y) in enumerate(train_loader):
        pos_u = torch.tensor(x)
        pos_v = torch.tensor(y)
        neg_v = torch.randint(0, dictionary_length, (10, negative_sample_length))
        # print(neg_v.shape)
        optimizer.zero_grad()
        pos_score, neg_score = model(pos_u, pos_v, neg_v)
        score = torch.cat([pos_score, neg_score.flatten()], dim=0)
        combined_len = len(pos_score) + len(neg_score)
        # add a column of ones to pos_u_data
        pos_u_data = torch.ones(len(pos_u), 1)
        neg_v_data = torch.zeros(len(neg_score.flatten()), 1)
        loss = criterion(score, concat_data) 
        
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
        if i % 1000 == 0:
            print(' Step [{}/{}], Loss: {:.4f}' 
                    .format(i+1, len(dataset)//batch_size, loss_sum/1000))
            loss_sum = 0


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])


/var/folders/xr/3pxy4p9914ld14l6mxvvknvw0000gn/T/ipykernel_29788/1079155030.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pos_u = torch.tensor(x)
/var/folders/xr/3pxy4p9914ld14l6mxvvknvw0000gn/T/ipykernel_29788/1079155030.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pos_v = torch.tensor(y)


RuntimeError: grad can be implicitly created only for scalar outputs

In [161]:
# get cosine similarity between two vectors


In [191]:
king_vec = model.get_embedding_from_word("king")
queen_vec = model.get_embedding_from_word("queen")



print(cos_sim(get_emb("god"), get_emb("dog")))



0.3412128


In [196]:
def subtract_vector(vector1,vector2):
    return get_emb(vector1) - get_emb(vector2)

def add_vector(vector1,vector2):
    return get_emb(vector1) + get_emb(vector2)

def cos_sim(word1, word2):
    vector1 = get_emb(word1)
    vector2 = get_emb(word2)
    return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

def get_emb(word):
    return model.get_embedding_from_word(word)

def get_closest_vector(vector):
    max = 0
    target = None
    for key,item in unique_dict.items():
        comparative = get_emb(key)
        comparison = cos_sim(vector, comparative)
        if comparison > max:
            max = comparison
            target = key

        
    return target


In [201]:
print(cos_sim("king", "queen"))

0.48564646


In [193]:
vector = subtract_vector("king", "man")
vector = get_emb("woman") + vector


In [194]:
print(get_closest_vector(vector))

woman


In [217]:
# save embeddings
path = "embeddings"
model.save_embedding(id_to_word, "outfile")

# To do
Clean up training code by adding epochs

Set up testing

run experiments

https://towardsdatascience.com/creating-word-embeddings-coding-the-word2vec-algorithm-in-python-using-deep-learning-b337d0ba17a8

https://www.deeplearningwizard.com/deep_learning/practical_pytorch/pytorch_logistic_regression/#step-3-building-model